In [ ]:
from typing import Union, List
from datasets import DatasetDict
import librosa
import numpy as np

from audio_dataset_creator.processors.audio_processor import AudioFile, ProcessedAudio
from audio_dataset_creator.audio_dataset_creator import PipelineStep

class ResampleTo44kHzStep(PipelineStep):
    def __init__(self, target_sr: int = 44100):
        self.target_sr = target_sr

    def process(self, input_data: Union[List[AudioFile], List[ProcessedAudio], DatasetDict]) -> DatasetDict:
        if not isinstance(input_data, DatasetDict):
            raise ValueError("ResampleTo44kHzStep expects a DatasetDict as input")

        def resample_audio(example):
            audio = np.array(example['audio'])
            sr = example['sampling_rate']
            if sr != self.target_sr:
                audio = librosa.resample(audio, orig_sr=sr, target_sr=self.target_sr)
            example['audio'] = audio.tolist()
            example['sampling_rate'] = self.target_sr
            return example

        return input_data.map(resample_audio)

    def __repr__(self) -> str:
        return f"ResampleTo44kHzStep(target_sr={self.target_sr})"


In [ ]:

from audio_dataset_creator import (
    AudioDatasetCreator,
    AudioDatasetCreatorConfig,
    AudioProcessor,
    DatasetCreator,
    DatasetSplitter,
    DatasetFilter
)
from audio_dataset_creator.processors.resample_step import ResampleTo44_1kHzStep

config = AudioDatasetCreatorConfig()
custom_pipeline = [
    AudioProcessor(),
    DatasetCreator(),
    ResampleTo44kHzStep(),  # Add the new step here
    DatasetSplitter(config.split_ratios),
    DatasetFilter(config.filter_conditions)
]

creator = AudioDatasetCreator(config=config, pipeline=custom_pipeline)